In [0]:
# Imports 

%tensorflow_version 2.x
import tensorflow as tf
from matplotlib import pyplot
from tensorflow.python import keras
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.models import Model,load_model
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.applications.resnet import ResNet50
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator,load_img 
from tensorflow.python.keras.layers import Dense,Conv2D, MaxPooling2D ,Activation, Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D


In [0]:
# Model Configuration
img_width, img_height = 224, 224  
train_data_dir = '/content/ProgDataRandom/prog_data_gaussian_random_85_10_5/training'
validation_data_dir = '/content/ProgDataRandom/prog_data_gaussian_random_85_10_5/validation'
test_data_dir = '/content/ProgDataRandom/prog_data_gaussian_random_85_10_5/testing'
nb_train_samples = 8500*5
nb_validation_samples = 1000*5
epochs = 50
batch_size = 50
reg=0.0001
learning_rate = 0.0001

In [0]:
# Setting up the image format/image shape
if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3)
 

In [0]:
# Created a model by using the "InceptionV3" as basemodel 
# and also appended few more layers to the base model

basemodel = VGG16(weights ="imagenet",include_top= False,input_shape=input_shape )
model = Sequential()
for layers in basemodel.layers:
    model.add(layers)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(512,kernel_initializer='he_normal',kernel_regularizer=l2(reg),bias_regularizer=l2(reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(256,kernel_initializer='he_normal',kernel_regularizer=l2(reg),bias_regularizer=l2(reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(128,kernel_initializer='he_normal',kernel_regularizer=l2(reg),bias_regularizer=l2(reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))


sgd = tf.keras.optimizers.SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)

model.compile( loss= keras.losses.categorical_crossentropy, 
               optimizer = sgd, 
               metrics=['accuracy'])


In [34]:
# The section is responsible for feeding the data during the training process
train_data_dir = '/content/ProgDataRandom/prog_data_gaussian_random_85_10_5/training'
validation_data_dir = '/content/ProgDataRandom/prog_data_gaussian_random_85_10_5/validation'
test_data_dir = '/content/ProgDataRandom/prog_data_gaussian_random_85_10_5/testing'
img_width, img_height = 224, 224
batch_size = 50

datagen = ImageDataGenerator( rescale = 1./255,
                              rotation_range=20,
                              zoom_range=0.15,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.15,
                              horizontal_flip=True,
                              fill_mode="nearest")   
 
test = ImageDataGenerator(rescale= 1./255)
train_generator = datagen.flow_from_directory(
                train_data_dir, 
                target_size =(img_width, img_height), 
                batch_size = batch_size,
                class_mode ='categorical',
                classes=['class_0','class_1','class_2','class_3','class_4'],
                shuffle = True
                ) 

validation_generator = datagen.flow_from_directory( 
                validation_data_dir, 
                target_size =(img_width, img_height), 
                batch_size = batch_size,
                class_mode ='categorical',
                classes=['class_0','class_1','class_2','class_3','class_4'],
                shuffle = True
                )
test_generator = test.flow_from_directory( 
                test_data_dir, 
                target_size =(img_width, img_height), 
                batch_size = batch_size,
                class_mode ='categorical',
                classes=['class_0','class_1','class_2','class_3','class_4'],
                shuffle = True
                )


Found 42500 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.
Found 2500 images belonging to 5 classes.


In [ ]:
# Started the Training proces

from tensorflow.python.keras.callbacks import ModelCheckpoint
filepath="weights_best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
history= model.fit(
    train_generator, 
    steps_per_epoch = nb_train_samples // batch_size, 
    epochs = epochs,
    validation_data = validation_generator, 
    validation_steps = nb_validation_samples // batch_size,
    shuffle = True,callbacks=callbacks_list
    )

model.save_weights('./model_saved_full.h5',overwrite= True)

In [0]:
# from tensorflow.python.keras.models import load_model from the my Google Drive
# The best trainned weights are stored in my GDrive as I was using colab for trainning purpose
# So, loading the best weight from my GDrive
# model = load_model("/content/drive/My Drive/VGG16.h5")
model.load_weights("/content/drive/My Drive/Diabetic Retinopathy VGG16 Weights and Model/weights_best_8068.hdf5")

In [36]:

# performing Evaluation
print('\n# Evaluate on test data')
results = model.evaluate(test_generator,batch_size = batch_size,steps = 2500 // batch_size)
print('test loss, test acc:', results)


# Evaluate on test data
50/50 [==============================] - 7s 139ms/step - loss: 0.8846 - accuracy: 0.7776
test loss, test acc: [0.8846393823623657, 0.7775999903678894]


In [0]:
# Testing the model with the best weight loaded 
# Examines the Testing set/ makes prediction of the testing set
import numpy as np
import os
import cv2
from sklearn.metrics import classification_report, confusion_matrix
# Imports 

%tensorflow_version 2.x
import tensorflow as tf
from matplotlib import pyplot
from tensorflow.python import keras
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.models import Model
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.applications.resnet import ResNet50
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator,load_img 
from tensorflow.python.keras.layers import Dense,Conv2D, MaxPooling2D ,Activation, Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D
#Y_pred = model.predict(test_generator,batch_size = batch_size,steps = 2500 // batch_size, use_multiprocessing=True)
label=[]
clas=[]
test_data_dir = '/content/ProgDataRandom/prog_data_gaussian_random_85_10_5/testing'
for j,i in enumerate(np.sort(os.listdir(test_data_dir))):
    p=os.path.join(test_data_dir,i)
    for image in os.listdir(p):

            inp = os.path.join(p,image)
            img = np.asarray(load_img(inp))/255
            img = np.reshape(img,[1,img_width,img_height,3])

            classes =model.predict_classes(img)[0]
            
            clas.append(classes)
            label.append(j)
    print(j," completed.")
          


0  completed.
1  completed.
2  completed.
3  completed.
4  completed.


In [0]:
print(label,clas,end="\n")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [0]:
# Generates the Confusion Matrix and other Reports
print('Confusion Matrix')
print(confusion_matrix(label, clas))
print('Classification Report')
target_names = ['class_0','class_1','class_2','class_3','class_4']
print(classification_report(label, clas, target_names=target_names))

Confusion Matrix
[[443  29  27   0   1]
 [ 76 403  20   0   1]
 [ 85  66 320  27   2]
 [ 42   6 164 287   1]
 [  1   1   7   0 491]]
Classification Report
              precision    recall  f1-score   support

     class_0       0.68      0.89      0.77       500
     class_1       0.80      0.81      0.80       500
     class_2       0.59      0.64      0.62       500
     class_3       0.91      0.57      0.71       500
     class_4       0.99      0.98      0.99       500

    accuracy                           0.78      2500
   macro avg       0.80      0.78      0.78      2500
weighted avg       0.80      0.78      0.78      2500



In [ ]:
# Use to visualize the result graphically

f1=pyplot.figure()
pyplot.plot(history.history['accuracy'], label='train_acc')
pyplot.plot(history.history['val_accuracy'], label='val_acc')
f2=pyplot.figure()
pyplot.plot(history.history['loss'], label='train_loss')
pyplot.plot(history.history['val_loss'], label='val_loss')
pyplot.legend()
pyplot.show()   
